<a href="https://colab.research.google.com/github/Sai145/2048/blob/master/BCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [4]:
!pip install -q findspark


In [5]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 65kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=f337fae6b1b7580c27f61b97fb1d91c5fd20e8d8b9b0534d98e9f50286df2e2e
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import *
# from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import *

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("BCG")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
spark

**Reading Data**

In [57]:
Charges=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Charges_use.csv")
Damages=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Damages_use.csv")
Endorse=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Endorse_use.csv")
PPerson=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Primary_Person_use.csv")
Restrict=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Restrict_use.csv")
Units=spark.read.options(header=True, inferSchema=True).csv("/content/drive/MyDrive/BCG/Units_use.csv")


**Cleaning Data**

In [58]:
Charges=Charges.dropDuplicates()
Damages=Damages.dropDuplicates()
Endorse=Endorse.dropDuplicates()
PPerson=PPerson.dropDuplicates()
Restrict=Restrict.dropDuplicates()
Units=Units.dropDuplicates()

**Analysis**-**1**

In [59]:
PPerson1=PPerson.filter((col("death_cnt")>=1) & (lower("PRSN_GNDR_ID")=='male'))
Analysis_1=PPerson1.select("Crash_ID").distinct()
Analysis_1.count()


180

**Analysis**-**2**

In [60]:
Charges2=Charges.select("CRASH_ID","UNIT_NBR").distinct()
Analysis_2=Units.join(Charges2,(Units.CRASH_ID==Charges2.CRASH_ID) & (Units.UNIT_NBR==Charges2.UNIT_NBR),"inner"
          ).filter(col("VEH_BODY_STYL_ID").isin("MOTORCYCLE","POLICE MOTORCYCLE"))
Analysis_2.count()

401

**Analysis-3**

In [61]:
Analysis_3=PPerson.filter(lower(col("PRSN_GNDR_ID"))=="female")
Analysis_3.groupBy("DRVR_LIC_STATE_ID").count().orderBy(col("count").desc()
).select("DRVR_LIC_STATE_ID").show(1)

+-----------------+
|DRVR_LIC_STATE_ID|
+-----------------+
|            Texas|
+-----------------+
only showing top 1 row



**Analysis-4**

In [62]:
Units4=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Units4=Units4.withColumn("Injury_Death",col("TOT_INJRY_CNT")+col("DEATH_CNT"))
Units4=Units4.groupby("VEH_MAKE_ID").sum("Injury_Death")
Analysis_4=Units4.withColumn("Filter",row_number().over(Window.orderBy(col("sum(Injury_Death)").desc())))
Analysis_4=Analysis_4.filter((col("Filter")>=5) & (col("Filter")<=15)).orderBy("Filter")
Analysis_4.select("VEH_MAKE_ID").show()

+-----------+
|VEH_MAKE_ID|
+-----------+
|     NISSAN|
|      HONDA|
|        GMC|
|    HYUNDAI|
|        KIA|
|       JEEP|
|   CHRYSLER|
|      MAZDA|
|         NA|
| VOLKSWAGEN|
|    PONTIAC|
+-----------+



**Analysis-5**

In [63]:

Units5=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Analysis_5=Units5.join(PPerson,(Units5.CRASH_ID==PPerson.CRASH_ID) & (Units5.UNIT_NBR==PPerson.UNIT_NBR),"inner"
          ).groupBy("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").count()
Analysis_5=Analysis_5.withColumn("Filter",row_number().over(Window.partitionBy("VEH_BODY_STYL_ID"
                        ).orderBy(col("count").desc())))
Analysis_5=Analysis_5.filter(col("Filter")==1)
Analysis_5.select("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").show()

+--------------------+-----------------+
|    VEH_BODY_STYL_ID|PRSN_ETHNICITY_ID|
+--------------------+-----------------+
|                 BUS|            BLACK|
|                  NA|            WHITE|
|                 VAN|            WHITE|
|              PICKUP|            WHITE|
|SPORT UTILITY VEH...|            WHITE|
|PASSENGER CAR, 4-...|            WHITE|
|          FIRE TRUCK|            WHITE|
|               TRUCK|            WHITE|
|             UNKNOWN|          UNKNOWN|
|           AMBULANCE|            WHITE|
|    POLICE CAR/TRUCK|            WHITE|
|          MOTORCYCLE|            WHITE|
|   YELLOW SCHOOL BUS|            BLACK|
|   POLICE MOTORCYCLE|            WHITE|
|PASSENGER CAR, 2-...|            WHITE|
|       TRUCK TRACTOR|            WHITE|
|      FARM EQUIPMENT|            WHITE|
|NEV-NEIGHBORHOOD ...|            WHITE|
|OTHER  (EXPLAIN I...|            WHITE|
|        NOT REPORTED|            WHITE|
+--------------------+-----------------+



**Analysis-6**

In [64]:
Units6=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Units6=Units6.filter(col("VEH_BODY_STYL_ID").like("%CAR%"))
Units6=Units6.filter(( upper(col("CONTRIB_FACTR_1_ID")).like("%ALCOHOL%") ) |
                    ( upper(col("CONTRIB_FACTR_2_ID")).like("%ALCOHOL%") ) |
                    ( upper(col("CONTRIB_FACTR_P1_ID")).like("%ALCOHOL%") ) )
PPerson6=PPerson.filter(col("DRVR_ZIP").isNotNull())
Analysis_6=Units6.join(PPerson6,(Units6.CRASH_ID==PPerson6.CRASH_ID) & (Units6.UNIT_NBR==PPerson6.UNIT_NBR),"inner"
          )
Analysis_6.groupBy("DRVR_ZIP").count().orderBy(desc("count"))
Analysis_6.select("DRVR_ZIP").show(5)

+--------+
|DRVR_ZIP|
+--------+
|   75050|
|   76135|
|   77032|
|   75090|
|   75088|
+--------+
only showing top 5 rows



**Analysis-7**

In [65]:
Units7=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Units7=Units7.filter(col("VEH_BODY_STYL_ID").like("%CAR%"))
Units7=Units7.filter(col("FIN_RESP_TYPE_ID").isin(["PROOF OF LIABILITY INSURANCE","INSURANCE BINDER","CERTIFICATE OF SELF-INSURANCE"]))

Units7=Units7.filter(( upper(col("VEH_DMAG_SCL_1_ID")).isin(["DAMAGED 5","DAMAGED 6","DAMAGED 7 HIGHEST"])) |
                    ( upper(col("VEH_DMAG_SCL_2_ID")).isin(["DAMAGED 5","DAMAGED 6","DAMAGED 7 HIGHEST"]) ) )

Analysis_7=Units7.join(Damages,Units7.CRASH_ID==Damages.CRASH_ID,"left_anti")
          

Analysis_7.select("CRASH_ID").distinct().count()

3862

**Analysis-8**

In [66]:
#licenced
PPerson8_3=PPerson.filter(~col("DRVR_LIC_CLS_ID").isin("NA","UNKNOWN","UNLICENSED"))


#speed
Charges8_3=Charges.filter(col("CHARGE").like('%SPEED%'))
Charges8_3=Charges8_3.dropDuplicates(["CRASH_ID","UNIT_NBR"])


#top 10 clours
Units8_2=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Units8_2=Units8_2.groupBy("VEH_COLOR_ID").count().filter(~col("VEH_COLOR_ID").isin('NA'))
Analysis_8_2=Units8_2.withColumn("Filter",row_number().over(Window.orderBy(col("count").desc())))
Analysis_8_2=Analysis_8_2.filter(col("Filter")<=10)

#top 25 cities 
Charges8_1=Charges.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Analysis_8_1=Charges8_1.join(PPerson,(PPerson.CRASH_ID==Charges8_1.CRASH_ID) & (PPerson.UNIT_NBR==Charges8_1.UNIT_NBR),"inner"
          )

Analysis_8_1=Analysis_8_1.groupBy("DRVR_LIC_STATE_ID").count().filter(~col("DRVR_LIC_STATE_ID").isin('NA',"Unknown"))

Analysis_8_1=Analysis_8_1.withColumn("Filter",row_number().over(Window.orderBy(col("count").desc())))
Analysis_8_1=Analysis_8_1.filter(col("Filter")<=25)
top25cities=['TX','MX','LA','NM','CA','FL','OK','AR','AZ','GA','IL','CO','MS','MO','TN','NC','KS','AL','OH','MI','NY','WA','VA','NV']


In [67]:
Units8=Units.dropDuplicates(["CRASH_ID","UNIT_NBR"])
Analysis8=Units8.join(PPerson8_3,(Units8.CRASH_ID==PPerson8_3.CRASH_ID) & (Units8.UNIT_NBR==PPerson8_3.UNIT_NBR),"inner"
              ).join(Charges8_3,(Units8.CRASH_ID==Charges8_3.CRASH_ID) & (Units8.UNIT_NBR==Charges8_3.UNIT_NBR),"inner"
             ).join(Analysis_8_2,Analysis_8_2.VEH_COLOR_ID==Units8.VEH_COLOR_ID,"inner"
             ).filter(col("VEH_LIC_STATE_ID").isin(['TX','MX','LA','NM','CA','FL','OK','AR','AZ','GA','IL','CO','MS','MO',
                                                    'TN','NC','KS','AL','OH','MI','NY','WA','VA','NV']))


In [68]:
Analysis8=Analysis8.groupBy("VEH_MAKE_ID").count()
Analysis8=Analysis8.withColumn("Filter",row_number().over(Window.orderBy(col("count").desc())))
Analysis8=Analysis8.filter(col("Filter")<=5).select("VEH_MAKE_ID")
Analysis8.show()

+-----------+
|VEH_MAKE_ID|
+-----------+
|       FORD|
|  CHEVROLET|
|     TOYOTA|
|      DODGE|
|      HONDA|
+-----------+

